In [2]:

# import cv2
# import numpy as np
# import os
# import glob
 
# # Defining the dimensions of checkerboard
# CHECKERBOARD = (6,9)
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# # Creating vector to store vectors of 3D points for each checkerboard image
# objpoints = []
# # Creating vector to store vectors of 2D points for each checkerboard image
# imgpoints = [] 
 

# # defining world points
# objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
# objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) # size of square in meters
# prev_img_shape = None
 

In [3]:
# # Extracting path of individual image stored in a given directory
# images = glob.glob('./iPhone16_calibration/*.JPG')
# for fname in images:
#     img = cv2.imread(fname)
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     # Find the chess board corners
#     # If desired number of corners are found in the image then ret = true
#     ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
#     """
#     If desired number of corner are detected,
#     we refine the pixel coordinates and display 
#     them on the images of checker board
#     """
#     if ret == True:
#         objpoints.append(objp)
#         # refining pixel coordinates for given 2d points.
#         corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
#         imgpoints.append(corners2)
 
#         # Draw and display the corners
#         img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
#     cv2.imshow('img',img)
#     cv2.waitKey(1)
#     cv2.destroyAllWindows()

# cv2.destroyAllWindows()
# cv2.waitKey(1)
 
# h,w = img.shape[:2]
 
# """
# Performing camera calibration by 
# passing the value of known 3D points (objpoints)
# and corresponding pixel coordinates of the 
# detected corners (imgpoints)
# """
# ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
# print("Camera matrix : \n")
# print(mtx)
# print("dist : \n")
# print(dist)
# print("rvecs : \n")
# print(rvecs)
# print("tvecs : \n")
# print(tvecs)

In [4]:
# print(ret)

In [5]:
# print(f"fx: {mtx[0][0]}, fy: {mtx[1][1]}")
# print(f"cx: {mtx[0][2]}, cy: {mtx[1][2]}")

# fx = mtx[0][0]
# fy = mtx[1][1]
# cx = mtx[0][2]
# cy = mtx[1][2]

In [6]:

import numpy as np
import cv2
import glob
import argparse

# termination criteriab
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def calibrate(dirpath, prefix, image_format, square_size, width=9, height=6):
    """ Apply camera calibration operation for images in the given directory path. """
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    if dirpath[-1:] == '/':
        dirpath = dirpath[:-1]

    images = glob.glob(dirpath+'/' + prefix + '*.' + image_format)

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (width, height), corners2, ret)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return [ret, mtx, dist, rvecs, tvecs]
def save_coefficients(mtx, dist, path):
    """ Save the camera matrix and the distortion coefficients to given path/file. """
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write("K", mtx)
    cv_file.write("D", dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()
def load_coefficients(path):
    """ Loads camera matrix and distortion coefficients. """
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode("K").mat()
    dist_matrix = cv_file.getNode("D").mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

In [7]:

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Camera calibration')
#     parser.add_argument('--image_dir', type=str, required=True, help='image directory path')
#     parser.add_argument('--image_format', type=str, required=True,  help='image format, png/jpg')
#     parser.add_argument('--prefix', type=str, required=True, help='image prefix')
#     parser.add_argument('--square_size', type=float, required=False, help='chessboard square size')
#     parser.add_argument('--width', type=int, required=False, help='chessboard width size, default is 9')
#     parser.add_argument('--height', type=int, required=False, help='chessboard height size, default is 6')
#     parser.add_argument('--save_file', type=str, required=True, help='YML file to save calibration matrices')

    # args = parser.parse_args()
ret, mtx, dist, rvecs, tvecs = calibrate("", "", "JPG", .025, 9, 6)
# save_coefficients(mtx, dist, args.save_file)
print("Calibration is finished. RMS: ", ret)

Calibration is finished. RMS:  2.1828099758145165


In [8]:
print(mtx)

[[3.08392653e+03 0.00000000e+00 1.98378736e+03]
 [0.00000000e+00 3.07897372e+03 1.49240130e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
